# Task
There is a function, which consumes string argument. It should validate that this string follows particular date-like pattern.

```
def process(argument: str) -> str:
    # fail if argument doesn't look like YYYY-mm-dd
    logic() 

```

## Common code and imports

In [16]:
import re
from datetime import datetime

## Approach 1 with assert

In [10]:
date_like_pattern = r'^\d{4}-\d{2}-\d{2}$'

def process(date_string: str) -> str:
    assert re.match(date_like_pattern, date_string)
    return "Success"

In [11]:
print(process("2022-12-12"))  # Success

Success


In [12]:
print(process("2022/12/12"))  # AssertionError

AssertionError: 

### Is there any problem?

In [14]:
! python3 custom_type_approach1.py

Success
Traceback (most recent call last):
  File "/Users/aburtsev/Code/PythonTricks/custom_type_approach1.py", line 10, in <module>
    print(process("2022/12/12"))  # AssertionError
  File "/Users/aburtsev/Code/PythonTricks/custom_type_approach1.py", line 5, in process
    assert re.match(date_like_pattern, date_string)
AssertionError


In [15]:
! python3 -O custom_type_approach1.py

Success
Success


Python's flag `-O` means optmization. Used quite often when code run in prod. It does a lot of optimizations, including ommiting all asserts

## Approach 2 with re 

In [17]:
def process(date_string: str) -> str:
    if not re.match(date_like_pattern, date_string):
        raise ValueError(f'Input param is not date: {date_string}')
    return "Success"

In [18]:
print(process("2022-12-12"))  # Success

Success


In [20]:
print(process("2022/12/12"))  # ValueError

ValueError: Input param is not date: 2022/12/12

### Downside of that approach?

 Some correct-looking string might be incorrect date

In [23]:
print(process("2022-13-99"))

Success


## Approach 3 with datetime

In [29]:
def validate_date_string(date_string: str) -> None:
    try:
        datetime.strptime(date_string, "%Y-%m-%d")
    except ValueError:
        raise ValueError(f'Input param is not date: {date_string}')

def process(date_string: str) -> str:
    validate_date_string(date_string)
    return "Success"

In [30]:
print(process("2022-12-12")) 

Success


In [31]:
print(process("2022-13-13"))

ValueError: Input param is not date: 2022-13-13

### Downside?

Actually, it's already good-enough. Potential downside, that we need to repeat it in each function and it's quite easy to forget to add it.

## Approach 4 with decorator

In [33]:
def validate_date(func):
    def wrapper(date_string):
        try:
            datetime.strptime(date_string, "%Y-%m-%d")
            return func(date_string)
        except ValueError:
            raise ValueError(f'Input param is not date: {date_string}')

    return wrapper

In [34]:
@validate_date
def process(date_string: str) -> str:
    return "Success"

In [35]:
print(process("2022-12-12"))

Success


In [36]:
print(process("2022-13-13"))

ValueError: Input param is not date: 2022-13-13

### Downside?

Only applicable to function with 1 param

## Approach 5 with configurable decorator

In [39]:
def validate_arg_is_date(arg_position):
    def internal_decorator(func):
        def wrapper(*args, **kwargs):
            try:
                datetime.strptime(args[arg_position], "%Y-%m-%d")
                return func(*args, **kwargs)
            except ValueError:
                raise ValueError(f'Input param is not date: {args[arg_position]}')

        return wrapper
    return internal_decorator

In [40]:
@validate_arg_is_date(1)
def process(first_param: int, date_string: str) -> str:
    return "Success"

In [42]:
print(process(10, "2022-12-12"))

Success


In [41]:
print(process(10, "2022-13-13"))

ValueError: Input param is not date: 2022-13-13

### Downside?

If we add new argument to function, we need not to forget to update index in decorators, but everyone always does that

## Approach 6 with custom Type via pydantic

In [44]:
! pip install pydantic

In [54]:
from pydantic import validate_arguments

In [55]:
class DateString(str):
    @classmethod
    def validate(cls, value: str) -> str:
        try:
            datetime.strptime(value, "%Y-%m-%d")
        except ValueError:
            raise ValueError(f'Input param is not date: {value}')
        return value

    @classmethod
    def __get_validators__(cls) -> 'CallableGenerator':
        yield cls.validate


@validate_arguments
def process(s3_prefix: str, date_string: DateString) -> str:
    return "Success"

In [56]:
print(process("2022-12-12"))

Success


In [57]:
print(process("2022-13-13"))

ValidationError: 1 validation error for Process
date_string
  Input param is not date: 2022-13-13 (type=value_error)

### Downside ?

I think it's the best way to solve that issue, but code might looks scary at first